In [1]:
from datasets import disable_caching, load_dataset

disable_caching()
pre_ds = load_dataset('brainer/korean-medicine-prescription')['train'].rename_columns({
    'STND_Y': '기준년도',
    'IDV_ID': '가입자 일련번호',
    'KEY_SEQ': '진료내역 일련번호',
    'SEQ_NO': '일련번호',
    'SEX': '성별코드',
    'AGE_GROUP': '연령대 코드',
    'SIDO': '시도코드',
    'RECU_FR_DT': '요양개시일자',
    'GNL_NM_CD': '약품 일반성분명 코드',
    'DD_MQTY_FREQ': '1회투약량',
    'DD_EXEC_FREQ': '1일투약량',
    'MDCN_EXEC_FREQ': '총투여일수또는실시횟수',
    'UN_COST': '단가',
    'AMT': '금액',
    'DATA_STD_DT': '데이터 기준일자'
})
pill_ds = load_dataset("brainer/pill_identification_data")['train']
medicode_ds = load_dataset("brainer/pill_identification_data", 'mdeication_code')['train']
pre_ds, pill_ds, medicode_ds

(Dataset({
     features: ['기준년도', '가입자 일련번호', '진료내역 일련번호', '일련번호', '성별코드', '연령대 코드', '시도코드', '요양개시일자', '약품 일반성분명 코드', '1회투약량', '1일투약량', '총투여일수또는실시횟수', '단가', '금액', '데이터 기준일자', '__index_level_0__'],
     num_rows: 32053871
 }),
 Dataset({
     features: ['품목일련번호', '품목명', '업소일련번호', '업소명', '성상', '큰제품이미지', '표시앞', '표시뒤', '의약품제형', '색상앞', '색상뒤', '분할선앞', '분할선뒤', '크기장축', '크기단축', '크기두께', '이미지생성일자(약학정보원)', '분류번호', '분류명', '전문일반구분', '품목허가일자', '제형코드명', '표기내용앞', '표기내용뒤', '표기이미지앞', '표기이미지뒤', '표기코드앞', '표기코드뒤', '변경일자', '사업자번호', 'Unnamed: 30'],
     num_rows: 25330
 }),
 Dataset({
     features: ['한글상품명', '업체명', '약품규격', '제품총수량', '제형구분', '포장형태', '품목기준코드', '품목허가일자', '전문_일반', '대표코드', '표준코드', '제품코드(개정후)', '일반명코드(성분명코드)', '비고', '취소일자', '양도양수적용(공고)일자', '양도양수종료일자', '일련번호생략여부', '일련번호생략사유', 'ATC코드', '특수관리약품구분', '의약품판독장비구분'],
     num_rows: 290750
 }))

In [2]:
from datasets import Dataset
import pandas as pd

# Merge with pre_ds['품목명'] medinfo_ds['품목명']
merge_ds = Dataset.from_pandas(pd.merge(
    pill_ds.to_pandas(), medicode_ds.to_pandas(), how='left', left_on='품목일련번호', right_on='품목기준코드'
).drop_duplicates('품목일련번호')).remove_columns(['Unnamed: 30', '__index_level_0__'])
merge_ds

Dataset({
    features: ['품목일련번호', '품목명', '업소일련번호', '업소명', '성상', '큰제품이미지', '표시앞', '표시뒤', '의약품제형', '색상앞', '색상뒤', '분할선앞', '분할선뒤', '크기장축', '크기단축', '크기두께', '이미지생성일자(약학정보원)', '분류번호', '분류명', '전문일반구분', '품목허가일자_x', '제형코드명', '표기내용앞', '표기내용뒤', '표기이미지앞', '표기이미지뒤', '표기코드앞', '표기코드뒤', '변경일자', '사업자번호', '한글상품명', '업체명', '약품규격', '제품총수량', '제형구분', '포장형태', '품목기준코드', '품목허가일자_y', '전문_일반', '대표코드', '표준코드', '제품코드(개정후)', '일반명코드(성분명코드)', '비고', '취소일자', '양도양수적용(공고)일자', '양도양수종료일자', '일련번호생략여부', '일련번호생략사유', 'ATC코드', '특수관리약품구분', '의약품판독장비구분'],
    num_rows: 25314
})

In [3]:
group_code_ds = Dataset.from_pandas(pd.DataFrame(
    medicode_ds.filter(lambda x: x['일반명코드(성분명코드)'] is not None).to_pandas().groupby('일반명코드(성분명코드)').apply(
        lambda x: list(x['품목기준코드'])))).rename_column('0', '품목기준코드')
group_code_ds

Filter:   0%|          | 0/290750 [00:00<?, ? examples/s]

Dataset({
    features: ['품목기준코드', '일반명코드(성분명코드)'],
    num_rows: 13267
})

In [4]:
group_code_ds.to_pandas()

,품목기준코드,일반명코드(성분명코드)
0,"[200002269, 200002269, 200002269, 200002269, 2...",100101ALQ
1,"[199001424, 199001424]",100101APD
2,"[200606991, 200606991, 200606991, 199501476, 1...",100102ACH
3,"[201901365, 201901365, 201901365, 201901365]",100102ALQ
4,"[199201988, 199201988, 199201988, 199201988]",100301ACH
...,...,...
13262,"[202302071, 202302071]",E75800ACH
13263,"[202302086, 202302086]",E75900ATB
13264,"[202302128, 202302128, 202302128]",E76000ACS
13265,"[202302140, 202302140]",E76100ACS


In [5]:
unique_code = pre_ds.unique('약품 일반성분명 코드')
len(unique_code)

2985

In [6]:
from tqdm.notebook import tqdm

# group_code_ds['일반명코드(성분명코드)']에 unique_code가 겹침 갯수확인
num_of_overlap = 0
for code in tqdm(unique_code):
    if code in group_code_ds['일반명코드(성분명코드)']:
        num_of_overlap += 1
num_of_overlap

  0%|          | 0/2985 [00:00<?, ?it/s]

2862

In [7]:
for row in group_code_ds.to_iterable_dataset():
    code, products = row['일반명코드(성분명코드)'], row['품목기준코드']
print(f"{code, products}")

('E76200ATB', [202302149, 202302149, 202302149])


In [8]:
pre_ds

Dataset({
    features: ['기준년도', '가입자 일련번호', '진료내역 일련번호', '일련번호', '성별코드', '연령대 코드', '시도코드', '요양개시일자', '약품 일반성분명 코드', '1회투약량', '1일투약량', '총투여일수또는실시횟수', '단가', '금액', '데이터 기준일자', '__index_level_0__'],
    num_rows: 32053871
})

In [9]:
import networkx as nx
from pyvis.network import Network
from tqdm.notebook import tqdm

# 그래프 생성
G = nx.Graph()
for row in tqdm(group_code_ds.to_iterable_dataset()):
    code, products = row['일반명코드(성분명코드)'], row['품목기준코드']
    for product in products:
        G.add_edge(code, product)

# pre_ds 데이터에서 '일반명코드(성분명코드)'의 중복 사용에 따라 edge 추가; 진료내역 일련번호가 같은 경우 weight 증가
for row in tqdm(pre_ds.to_iterable_dataset(), total=len(pre_ds)):
    code, seq_no = row['약품 일반성분명 코드'], row['진료내역 일련번호']
    for row2 in pre_ds.filter(lambda x: x['진료내역 일련번호'] == seq_no, num_proc=6).to_iterable_dataset():
        code2 = row2['약품 일반성분명 코드']
        if code != code2:
            G.add_edge(code, code2)
        else:
            if G.has_edge(code, code2):
                G[code][code2]['weight'] += 1
            else:
                G.add_edge(code, code2, weight=1)

0it [00:00, ?it/s]

Process SpawnPoolWorker-2:
Traceback (most recent call last):
  File "/Users/brainer/anaconda3/envs/PillDetectGraph/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/brainer/anaconda3/envs/PillDetectGraph/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/brainer/anaconda3/envs/PillDetectGraph/lib/python3.8/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Users/brainer/anaconda3/envs/PillDetectGraph/lib/python3.8/multiprocessing/queues.py", line 358, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'process_shard' on <module '__main__' (built-in)>
Process SpawnPoolWorker-3:
Traceback (most recent call last):
  File "/Users/brainer/anaconda3/envs/PillDetectGraph/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/brainer/anaconda3/envs/PillDetectGraph/lib/python3.8/multipro

KeyboardInterrupt: 

In [ ]:
# Pyvis 네트워크 생성
net = Network(notebook=True)
net.from_nx(G)

# 네트워크 시각화
net.show('graph.html')

In [ ]:
pd.DataFrame(net.get_edges())